In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.linalg import sqrtm
from copy import deepcopy
from datetime import datetime
import time

In [2]:
## load data
train = pd.read_csv('products.csv')
test = pd.read_csv('sampleSubmission.csv')

C:\Anaconda3\envs\aind-dog-copy\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
train['promotion_description'].fillna('no_promo', inplace=True)
train['Gender'].fillna('no_gender', inplace=True)
train['State'].fillna('no_state', inplace=True)
train['PinCode'].fillna(-1, inplace=True)
train['DOB'].fillna("1", inplace=True)

In [4]:
#handling missing data for product_code
train = train[np.isfinite(train['product_code'])]

In [5]:
cnt_srs = train['product_code'].astype('int64').value_counts().reset_index().head(20)
#cnt_srs = products['product_code'].value_counts().reset_index()
cnt_srs.columns = ['product_code', 'frequency_count']
cnt_srs

,product_code,frequency_count
0,300776411,60994
1,300776410,58437
2,108037568,51338
3,300785148,41500
4,108100382,35259
5,300785150,34794
6,300840018,31149
7,108100362,27963
8,108005676,27727
9,300111517,25360


In [6]:
prod_index = []
for i in cnt_srs['product_code']:
    prod_index.append(i)

In [7]:
prod_index

[300776411,
 300776410,
 108037568,
 300785148,
 108100382,
 300785150,
 300840018,
 108100362,
 108005676,
 300111517,
 108100306,
 108000707,
 300776409,
 300781593,
 108100308,
 300481673,
 108100294,
 108100290,
 108100352,
 300070102]

In [8]:
product_joined = pd.merge(train, cnt_srs, on='product_code', how='inner')
products_less = product_joined[['customerID', 'Gender', 'State', 'transactionDate', 'product_code']].reset_index()
products_less.drop('index', axis=1, inplace=True)
products_to_join = pd.DataFrame(train['customerID']).reset_index()
products_to_join.drop('index', axis=1, inplace=True)

In [9]:
# Join the dummy variables to the main dataframe
train = pd.merge(products_to_join, products_less.drop_duplicates(subset=['customerID'], keep='last'), on='customerID', how='left')
train.head()

,customerID,Gender,State,transactionDate,product_code
0,BBID_2041,male,no_state,2016-03-20,1.081e+08
1,BBID_2041,male,no_state,2016-03-20,1.081e+08
2,BBID_2041,male,no_state,2016-03-20,1.081e+08
3,BBID_2041,male,no_state,2016-03-20,1.081e+08
4,BBID_2041,male,no_state,2016-03-20,1.081e+08


In [10]:
train.dropna(inplace=True)

In [11]:
prod_dict = {'_'}

train = pd.get_dummies(train, columns=['product_code'], prefix=prod_dict)
train.columns=train.columns.str.replace('_','')
train.reset_index(drop=True).head()

,customerID,Gender,State,transactionDate,108000707.0,108005676.0,108037568.0,108100290.0,108100294.0,108100306.0,...,300070102.0,300111517.0,300481673.0,300776409.0,300776410.0,300776411.0,300781593.0,300785148.0,300785150.0,300840018.0
0,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
train.columns

Index(['customerID', 'Gender', 'State', 'transactionDate', '108000707.0',
       '108005676.0', '108037568.0', '108100290.0', '108100294.0',
       '108100306.0', '108100308.0', '108100352.0', '108100362.0',
       '108100382.0', '300070102.0', '300111517.0', '300481673.0',
       '300776409.0', '300776410.0', '300776411.0', '300781593.0',
       '300785148.0', '300785150.0', '300840018.0'],
      dtype='object')

In [ ]:
df_train = train.copy()

In [ ]:
pd.set_option('display.max_columns', 600)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
df_train.columns[4:]

In [ ]:
product_col = ['100105505.0', '108000537.0', '108000568.0', '108000707.0',
       '108001125.0', '108001127.0', '108003448.0', '108003451.0',
       '108004423.0', '108004880.0', '108004977.0', '108005676.0',
       '108005681.0', '108008866.0', '108010225.0', '108015124.0',
       '108015133.0', '108017242.0', '108017245.0', '108018729.0',
       '108018730.0', '108018925.0', '108020429.0', '108030063.0',
       '108030113.0', '108032048.0', '108037499.0', '108037504.0',
       '108037568.0', '108043278.0', '108100137.0', '108100143.0',
       '108100145.0', '108100183.0', '108100193.0', '108100243.0',
       '108100253.0', '108100267.0', '108100278.0', '108100288.0',
       '108100290.0', '108100294.0', '108100296.0', '108100297.0',
       '108100298.0', '108100302.0', '108100306.0', '108100308.0',
       '108100319.0', '108100328.0', '108100330.0', '108100331.0',
       '108100334.0', '108100335.0', '108100340.0', '108100352.0',
       '108100361.0', '108100362.0', '108100382.0', '300028156.0',
       '300070102.0', '300088564.0', '300111517.0', '300143077.0',
       '300144423.0', '300150379.0', '300157247.0', '300177372.0',
       '300317078.0', '300326493.0', '300347917.0', '300361324.0',
       '300374464.0', '300481673.0', '300481740.0', '300481741.0',
       '300497842.0', '300528552.0', '300616595.0', '300647581.0',
       '300676075.0', '300680079.0', '300685217.0', '300776409.0',
       '300776410.0', '300776411.0', '300781593.0', '300785147.0',
       '300785148.0', '300785150.0', '300808923.0', '300840018.0',
       '300942704.0', '300974316.0', '300974360.0', '500096181.0',
       '1000010411.0', '1000058092.0', '1000336252.0', '1000609658.0']

In [ ]:
df_train.head()

In [ ]:
month = 6

df_train_curr = df_train.loc[df_train['transactionDate']=='2016-06-28',:]
df_train_5 = df_train.loc[df_train['transactionDate']=='2016-05-28', product_col+['customerID']]
df_train_4 = df_train.loc[df_train['transactionDate']=='2016-04-28', product_col+['customerID']]
df_train_3 = df_train.loc[df_train['transactionDate']=='2016-03-28', product_col+['customerID']]
df_train_2 = df_train.loc[df_train['transactionDate']=='2016-02-28', product_col+['customerID']]
df_train_1 = df_train.loc[df_train['transactionDate']=='2016-01-28', product_col+['customerID']]

In [ ]:
dfm = pd.merge(df_train_curr,df_train_5, how='left', on=['customerID'], suffixes=('', '_5'))
dfm = pd.merge(dfm,df_train_4, how='left', on=['customerID'], suffixes=('', '_4'))
dfm = pd.merge(dfm,df_train_3, how='left', on=['customerID'], suffixes=('', '_3'))
dfm = pd.merge(dfm,df_train_2, how='left', on=['customerID'], suffixes=('', '_2'))
dfm = pd.merge(dfm,df_train_1, how='left', on=['customerID'], suffixes=('', '_1'))

In [ ]:
dfm.head()

In [ ]:
prevcols = [col for col in dfm.columns if '.0_'+str(month-1) in col]
currcols = [col for col in dfm.columns if '.0' in col[-5:]]

In [ ]:
currcols = currcols[0:100]

In [ ]:
all_product_col = [col for col in dfm.columns if '.0' in col]

for col in all_product_col:
    dfm[col].fillna(0, inplace=True)

In [ ]:
for col in currcols:
    dfm[col] = dfm[col] - dfm[col+'_'+str(month-1)]
    dfm[col] = dfm[col].apply(lambda x: max(x,0))

In [ ]:
dfm = dfm[dfm[currcols].sum(axis=1) >0]


In [ ]:
print(dfm[currcols].sum(axis = 1).value_counts())


In [ ]:
dfm

In [ ]:
dfm = dfm.reset_index(drop=True) 
print(dfm.shape)

In [ ]:
data = []

for index, row in dfm.iterrows():
    if index%1000 == 0:
        print(index)
        print('finish: ', time.strftime('%a %H:%M:%S'))
    if row[currcols].sum() > 0:
        for i,col in enumerate(currcols):
            if row[col] == 1:
                row['target'] = float(currcols.index(col))
                data.append(list(row))
                
df_new = pd.DataFrame(data, columns = list(dfm.columns.values) + ['target'])

In [ ]:
df_new.drop(currcols+['transactionDate'], axis=1, inplace=True)

In [ ]:
df_new.shape


In [ ]:
print(df_new['target'].value_counts())


In [ ]:
df_new.to_csv('cleaned_data/DataMulticlass_'+ str(month) + '_withpast2.csv', index=False)
